# IMPORTS

In [21]:
import re
import holidays
from collections import Counter
import sklearn as skl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.metrics import classification_report,plot_confusion_matrix, accuracy_score
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
import warnings
warnings.filterwarnings("ignore")

In [22]:
# leitura
incidentes = pd.read_csv('../input/training_data.csv')
teste = pd.read_csv('../input/test_data.csv')

In [23]:
# remoção de colunas desnecessárias
incidentes=incidentes.drop(['city_name'],axis=1)
incidentes=incidentes.drop(['avg_precipitation'],axis=1)
teste=teste.drop(['city_name'],axis=1)
teste=teste.drop(['avg_precipitation'],axis=1)

In [24]:
# transformação para numéricas
incidentes["avg_rain"] = incidentes['avg_rain'].replace({'Sem Chuva' : 0, 'chuva fraca' : 1, 'chuva moderada' : 2, 'chuva forte' : 3}).astype(int)
incidentes["magnitude_of_delay"] = incidentes['magnitude_of_delay'].replace({'UNDEFINED' : 0, 'MODERATE' : 1, 'MAJOR' : 2}).astype(int)
incidentes['luminosity'] = incidentes['luminosity'].replace(['DARK','LOW_LIGHT','LIGHT'],[0,1,2]).astype(int)
incidentes['record_date'] = pd.to_datetime(incidentes['record_date'])
incidentes["incidents"] = incidentes['incidents'].replace({'None' : 0, 'Low' : 1, 'Medium' : 2, 'High' : 3, 'Very_High' : 4}).astype(int)

teste["avg_rain"] = teste['avg_rain'].replace({'Sem Chuva' : 0, 'chuva fraca' : 1, 'chuva moderada' : 2, 'chuva forte' : 3}).astype(int)
teste["magnitude_of_delay"] = teste['magnitude_of_delay'].replace({'UNDEFINED' : 0, 'MODERATE' : 1, 'MAJOR' : 2}).astype(int)
teste['luminosity'] = teste['luminosity'].replace(['DARK','LOW_LIGHT','LIGHT'],[0,1,2]).astype(int)

In [25]:
## MÉTODO 1
# Se for ',' vamos remover, é bem provável que tenha sido um erro (=0)
# Para as restantes, vamos colocar as estradas diferentes afetadas
entries = []
print("Starting...")
for row in incidentes['affected_roads']:
    if row == ',':
        # Substituir por '0'
        entry = 0
    else:
        words = str(row).split(',') # array com as estradas
        words_no_empty = list(filter(lambda x: x != '', words))
        values, counts = np.unique(words_no_empty, return_counts=True) # lista de valores únicos
        entry = values.size
    entries.append(entry)
incidentes['affected_roads_number'] = entries
entries = []
for row in teste['affected_roads']:
    if row == ',':
        # Substituir por '0'
        entry = 0
    else:
        words = str(row).split(',') # array com as estradas
        words_no_empty = list(filter(lambda x: x != '', words))
        values, counts = np.unique(words_no_empty, return_counts=True) # lista de valores únicos
        entry = values.size
    entries.append(entry)
teste['affected_roads_number'] = entries
print("Done.")

Starting...
Done.


In [26]:
# remover a coluna tratada
incidentes=incidentes.drop(['affected_roads'],axis=1)
teste=teste.drop(['affected_roads'],axis=1)

In [27]:
# tratar o record_time
# treino
incidentes['record_date'] = pd.to_datetime(incidentes['record_date'])
incidentes['day_of_month'] = incidentes['record_date'].dt.day
incidentes['week_day'] = incidentes['record_date'].dt.weekday
incidentes['month'] = incidentes['record_date'].dt.month
incidentes['year'] = incidentes['record_date'].dt.year
incidentes['hour'] = incidentes['record_date'].dt.hour
incidentes['minute'] = incidentes['record_date'].dt.minute
incidentes['day_of_year'] = incidentes['record_date'].dt.dayofyear

# teste
teste['record_date'] = pd.to_datetime(teste['record_date'])
teste['day_of_month'] = teste['record_date'].dt.day
teste['week_day'] = teste['record_date'].dt.weekday
teste['month'] = teste['record_date'].dt.month
teste['year'] = teste['record_date'].dt.year
teste['hour'] = teste['record_date'].dt.hour
teste['minute'] = teste['record_date'].dt.minute
teste['day_of_year'] = teste['record_date'].dt.dayofyear


In [28]:
# remover essa coluna
incidentes.drop('minute', axis=1, inplace=True)
incidentes.drop('year', axis=1, inplace=True)
teste.drop('minute', axis=1, inplace=True)
teste.drop('year', axis=1, inplace=True)

In [29]:
incidentes.head()

,magnitude_of_delay,delay_in_seconds,record_date,luminosity,avg_temperature,avg_atm_pressure,avg_humidity,avg_wind_speed,avg_rain,incidents,affected_roads_number,day_of_month,week_day,month,hour,day_of_year
0,0,0,2021-03-15 23:00:00,0,12.0,1013.0,70.0,1.0,0,0,0,15,0,3,23,74
1,0,385,2021-12-25 18:00:00,0,12.0,1007.0,91.0,1.0,0,0,1,25,5,12,18,359
2,0,69,2021-03-12 15:00:00,2,14.0,1025.0,64.0,0.0,0,1,0,12,4,3,15,71
3,2,2297,2021-09-29 09:00:00,2,15.0,1028.0,75.0,1.0,0,4,3,29,2,9,9,272
4,0,0,2021-06-13 11:00:00,2,27.0,1020.0,52.0,1.0,0,3,1,13,6,6,11,164


## Cenário 1 - Feature Engeneering
* Vamos extrair algumas features que podem ter interesse:
    * Estação do Ano;
    * Fim de semana;
    * Feriado;
* A ideia é depois relacionar com as métricas e verificar se o modelo melhorou

In [30]:
# obter estação do ano para cada dia
def get_season(df):
    seasons = []
    for index, row in df.iterrows():
        # Access the data in each column by column name
        month = row['month']
        day = row['day_of_month']
    
        # Determine the season based on the month and day of the month
        if month in [3, 4, 5]:
            # For March, April, and May, use the day to determine the season
            if day < 21:
                season = 0
            else:
                season = 1
        elif month in [6, 7, 8]:
            season = 1
        elif month in [9, 10, 11]:
            # For September, October, and November, use the day to determine the season
            if day < 21:
                season = 2
            else:
                season = 3
        else:
            season = 3
        seasons.append(season)
    return seasons

In [31]:
incidentes['season'] = get_season(incidentes)
teste['season'] = get_season(teste)

In [32]:
# determina se é ou não um dia de férias
def is_holidays(df):
    pt_holidays = holidays.CountryHoliday("PT")
    df["is_holiday"] = 0

    for i, row in df.iterrows():
        date_field = row["record_date"]
        if date_field in pt_holidays:
            df.at[i, "is_holiday"] = 1
    return df

In [33]:
incidentes = is_holidays(incidentes)
teste = is_holidays(teste)

In [34]:
# determina se é fim de semana ou não
def is_weekend(df):
    df["is_weekend"] = 1
    for i, row in df.iterrows():
        date_field = row["week_day"]
        if date_field in [0, 4]:
            df["is_weekend"] = 0
    return df

In [35]:
incidentes = is_weekend(incidentes)
teste = is_weekend(teste)

In [36]:
# podemos remover a coluna
incidentes.drop('record_date', axis=1, inplace=True)
# podemos remover a coluna
teste.drop('record_date', axis=1, inplace=True)

In [37]:
incidentes.head()

,magnitude_of_delay,delay_in_seconds,luminosity,avg_temperature,avg_atm_pressure,avg_humidity,avg_wind_speed,avg_rain,incidents,affected_roads_number,day_of_month,week_day,month,hour,day_of_year,season,is_holiday,is_weekend
0,0,0,0,12.0,1013.0,70.0,1.0,0,0,0,15,0,3,23,74,0,0,0
1,0,385,0,12.0,1007.0,91.0,1.0,0,0,1,25,5,12,18,359,3,1,0
2,0,69,2,14.0,1025.0,64.0,0.0,0,1,0,12,4,3,15,71,0,0,0
3,2,2297,2,15.0,1028.0,75.0,1.0,0,4,3,29,2,9,9,272,3,0,0
4,0,0,2,27.0,1020.0,52.0,1.0,0,3,1,13,6,6,11,164,1,0,0


### Modelo
* Vamos verificar se há alterações nas métricas

In [38]:
#divisão do target
x = incidentes.drop(['incidents'], axis=1)
y = incidentes['incidents']

In [39]:
# função que constrói os modelos, apresentando as métricas
def train_and_predict(X, y, teste):
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

    models = [RandomForestClassifier(n_estimators=100, max_features="auto", random_state=42),
             LGBMClassifier(boosting_type='gbdt', learning_rate=0.1, n_estimators=100, random_state=42),
             GradientBoostingClassifier(learning_rate=0.1, n_estimators=100, random_state=42),
             DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=20, random_state=42),
             ExtraTreesClassifier(criterion='gini', max_depth=20, random_state=42),
             KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto')]
             
    reports=[]
    acc = []
    pred = []
    names = ['RF','LGB','GB','DT','ET','KN']
    
    for i, model in enumerate(models):
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions on the test set
        y_pred = model.predict(X_test)
        report = classification_report(y_test, y_pred)
        reports.append(report)
        accuracy = accuracy_score(y_test, y_pred)
        acc.append(accuracy)
        test_predictions = model.predict(teste)
        pred.append(test_predictions)

    i = 0   
    #Print the reports side by side
    for i, report in enumerate(reports):
        print("Model {} {}:\n{}".format(i + 1, names[i], report))
        print("Accuracy: {:.5f}".format(acc[i]))
        print("=" * 50)
    return pred

In [40]:
preds = train_and_predict(x,y,teste)

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f239c13a8b0>
Traceback (most recent call last):
  File "/home/purp/anaconda3/envs/dataset_COMPETICAO/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/purp/anaconda3/envs/dataset_COMPETICAO/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/purp/anaconda3/envs/dataset_COMPETICAO/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/purp/anaconda3/envs/dataset_COMPETICAO/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes call

Model 1 RF:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       517
           1       0.87      0.88      0.87       189
           2       0.86      0.81      0.84       144
           3       0.91      0.89      0.90       265
           4       0.90      0.94      0.92       135

    accuracy                           0.92      1250
   macro avg       0.90      0.90      0.90      1250
weighted avg       0.92      0.92      0.92      1250

Accuracy: 0.92480
Model 2 LGB:
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       517
           1       0.87      0.88      0.88       189
           2       0.85      0.81      0.83       144
           3       0.90      0.88      0.89       265
           4       0.90      0.93      0.91       135

    accuracy                           0.92      1250
   macro avg       0.90      0.90      0.90      1250
weighted avg       0.92      0.92 

### Comparação

| **Modelo**                          | **Antes** | **Depois** | **Obs**      |
|-------------------------------------|-----------|------------|--------------|
| **K Neighbors Classifier**          | 0.8312    | 0.83120    | **Manteve**  |
| **Extra Trees Classifier**          | 0.8968    | 0.89760    | **Melhorou** |
| **Gradient Boosting Classifier**    | 0.8984    | 0.90000    | **Melhorou** |
| **Decision Tree Classifier**        | 0.9080    | 0.90640    | Piorou       |
| **Random Forest Classifier**        | 0.9168    | 0.92480    | **Melhorou** |
| **Light Gradient Boosting Machine** | 0.9232    | 0.92240    | Piorou       |

Pensamos depois, em alterar o método da coluna affected_roads. Testaremos novamente para a feature com esse método (notebook 2.1)